In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import os

# 1. Twitter tennis datasets

We compiled two separate tweet collections, `RG17` for **Roland-Garros 2017**, the French
Open Tennis Tournament, and `uo17` for **US Open 2017**, the United States Open Tennis
Championship.

## Data collection

- We gathered data with the Twitter Search API, by using the following two separate sets of keywords:

```
{@rolandgarros, #RolandGarros2017,
#rolandgarros2017, #RolandGarros, #rolandgarros,
#FrenchOpen, #frenchopen, #RG17, #rg17}

{#usopen, #Usopen, #UsOpen, #USOPEN,
#usopen17, #UsOpen17, #Usopen2017, @usopen,
#WTA, #wta, #ATP, #atp, @WTA, @ATPWorldTour,
#Tennis, #tennis, #tenis, #Tenis}
```

**NOTE:** retweets were excluded from these datasets

## Event timeline

- **Qualifications:**
   - RG17: 2017-05-22 -> 2017-05-26 (we collected data only from 2017-05-24)
   - UO17: 2017-08-22 -> 2017-08-25
- **Days without tournaments:**
   - RG17: 2017-05-27
   - UO17: 2017-08-26 -> 2017-08-27
- **Main tournaments:** used for supervised model evaluation
   - RG17: 2017-05-28 -> 2017-06-11 (15 days)
   - UO17: 2017-08-28 -> 2017-09-10 (14 days)

## Dataset preparation

In [ ]:
from online_node2vec.data.tennis_handler import download_data_set
data_dir = "../data"
dataset_id = "rg17"

In [ ]:
preproc_dir, handler = download_data_set(data_dir, dataset_id)
print("Path for preprocessed data:", preproc_dir)

### Edge stream

- **Nodes:** Twitter accounts
- **Edges:** 
   - @-mentions between accounts
   - timestamp is extracted from the tweet

In [ ]:
edges = np.loadtxt(os.path.join(preproc_dir, "edges.csv"), delimiter="|", max_rows=5)
print(edges)

Number of nodes and edges in the RG17 mention graph. 
- During the qualifiers the number of interactions is low. 
- Then user activity increases as the championship starts from May 28  
- Activity burst due to Men’s Singles finals on June 7–9–11

In [ ]:
fig = handler.visualize(kind="graph", figsize=(8,5))
fig.axes[0].set_title("RG17")

### Official event schedule

In [ ]:
handler.schedule.head()

### Tennis player accounts on Twitter

- The total number of professional participants for Roland-Garros is 698. Unfortunately, many of the players have no Twitter accounts.

- We assigned players to accounts by the Twitter Search API’s people endpoint; however, the API was sometimes unable to identify the accounts of the active players.

In [ ]:
fig = handler.visualize(kind="players", figsize=(8,5))
fig.axes[0].set_title("RG17")

### Node relevance labels

- An account in the Twitter mention graph is **relevant** if it corresponds to a tennis player that participated in the tournaments of the given day
- Binary node relevance labels change from day to day based on the official schedule of the tournaments.
- Supervised evaluation for the main tournaments: 2017-05-28 -> 2017-06-11 (15 days)

In [ ]:
players_in_finals = np.loadtxt(os.path.join(preproc_dir, "labels_18.csv"), delimiter=" ")
print("Found relevant players on the last day:", len(players_in_finals))

In [ ]:
print(players_in_finals[:5])

# 2. Online node embeddings

## i.) Initialize models

Our online node embedding algorithms have two components:

- **updater:** sample node pairs for representation learning
- **learner:** online Word2Vec implementation

In [ ]:
from online_node2vec.online.walk_sampling import StreamWalkUpdater
from online_node2vec.online.walk_sampling import SecondOrderUpdater
from online_node2vec.online.w2v_learners import OnlineWord2Vec

### a.) StreamWalk

**Updater parameters:**

- half_life : Half-life in seconds for time decay
- max_len : Maximum length of the sampled temporal random walks
- beta : Damping factor for long paths
- k: Number of sampled walks for each edge update

In [ ]:
sw_updater = StreamWalkUpdater(
    half_life=3*3600, 
    max_len=2, 
    beta=0.9, 
    k=4,
)    

**Learner parameters:**

- embedding_dims :Dimensions of the representation
- lr_rate : Learning rate
- neg_rate: Negative rate
- mirror: Feed sampled node pairs in both order to the learner
- exportW1: bool

In [ ]:
dimensions = 128

In [ ]:
sw_learner = OnlineWord2Vec(
    embedding_dims=dimensions, 
    lr_rate=0.035, 
    neg_rate=10, 
    mirror=False, 
    exportW1=False, 
)

### b.) Online second order similarity

**Updater parameters:**

- half_life : Half-life in seconds for time decay
- num_hash : Number of hash functions to use for similarity approximation
- hash_gen : Hash function generator class.

In [ ]:
from online_node2vec.online.hash_utils import ModHashGenerator
so_updater = SecondOrderUpdater(
    half_life=12*3600,
    num_hash=20,
    hash_generator=ModHashGenerator()
)

**Learner parameters:**

- uniform_ratio: Fraction of uniform random negative samples. Remaining negative samples are chosen from past positive training instances.

In [ ]:
so_learner = OnlineWord2Vec(
    embedding_dims=dimensions, 
    lr_rate=0.01, 
    neg_rate=20, 
    uniform_ratio=0.8,
)

## ii.) Run models

**Experiment setup:**
- node representations are
    - calculated for the first 3 days
    - exported in every 6 hours
- the total elapsed time and Word2Vec training time are shown in seconds

In [ ]:
result_folder = "../results/"
total_days = 3
delta_time = 6*3600
root_dir = "%s/%s/features_%s/delta_%i" % (result_folder, dataset_id, 0, delta_time)

from online_node2vec.data.tennis_handler import load_edge_data
edge_data, start_time, end_time = load_edge_data(data_dir, dataset_id, total_days)

#### Calculate and export StreamWalk representations 

In [ ]:
# Execution takes more time!
from online_node2vec.online.online_node2vec_models import LazyNode2Vec
streamwalk = LazyNode2Vec(sw_updater, sw_learner, is_decayed=True)
streamwalk_output_dir = streamwalk.run(edge_data, delta_time, root_dir, start_time=start_time, end_time=end_time)

#### Calculate and export online second order similarity representations 

In [ ]:
# Execution takes more time!
second_order = LazyNode2Vec(so_updater, so_learner, is_decayed=True)
second_order_output_dir = second_order.run(edge_data, delta_time, root_dir, start_time=start_time, end_time=end_time)

## iii.) Observe node representations over time

### Load label information

In [ ]:
from online_node2vec.data.tennis_handler import get_data_info
gen_id_to_account, player_labels = get_data_info(os.path.join(data_dir, "%s_preprocessed" % dataset_id))

### Load node representations

In [ ]:
print(streamwalk_output_dir)

In [ ]:
model_dirs = {
    "so":second_order_output_dir,
    "sw":streamwalk_output_dir
}

In [ ]:
from online_node2vec.data.n2v_embedding_handler import load_n2v_features
feature_sets = {}
for model_id in ["sw","so"]:
    feature_sets[model_id] = load_n2v_features(
        model_dirs[model_id], 
        delta_time, 
        total_days, 
        player_labels, 
        verbose=False
    )

### Animation: Tennis player account embedding

- **Nodes:** Twitter accounts of tennis players
    - **red:** players who play on the given day
    - **blue:** player who doesn't play on the given day
    - colors change from day to day (after every 4 snapshots)
    
- **Representations:**
    - initialized uniformly at random in a 128-dimensional space (close to the origo in snapshot=0)
    - updated online as @-mention arrive in the edge stream (we only show the first two coordinates)
    - performance is considered better when red nodes are mapped close to each other
- **Results:**
    - StreamWalk (with 3 hours half-time) efficiently capture daily players by the end of each day
    - Online second order similarity (with longer half-life) can also map daily players close to each other in the embedded space:
        - Try to find red node groups related to Women's or Men's Singles around snapshot 7

In [ ]:
dates = ["2017-05-28","2017-05-29","2017-05-30"]

In [ ]:
from notebook_utils import prepare_animation
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

fig_sw, embedding_sw = prepare_animation(
    handler, 
    player_labels, 
    feature_sets["sw"], 
    delta_time, 
    "StreamWalk", 
    dates
)
fig_so, embedding_so = prepare_animation(
    handler, 
    player_labels, 
    feature_sets["so"], 
    delta_time, 
    "Online second order", 
    dates
)

In [ ]:
fig_sw

In [ ]:
fig_so

## iv.) Task: similarity search

**active account:** account `n` is active on the given day, if it belongs to a tennis player who participated in a completed, canceled, or resumed game.

We use [NDCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (Al-Maskari et al. 2007) to evaluate how other active accounts are similar
to a selected one. NDCG is a measure for ranked lists that assigns higher score if active
accounts appear with higher rank in the similarity list. 


In our experiments, we compute
the average of the **NDCG@100** for the active accounts as query nodes to measure the
performance of a single model in any given snapshot. **Dot product** is used to measure the similarity of node representations.

### a.) Single model performance

In [ ]:
from online_node2vec.evaluation.ndcg_computer import parallel_eval_ndcg
metric = "-dot"
max_threads = 2
model_performance = {}
for model_id in ["sw","so"]:
    model_performance[model_id] = pd.concat(parallel_eval_ndcg(
        feature_sets[model_id], 
        gen_id_to_account, 
        metric,
        n_threads=max_threads
    ))

In [ ]:
sw_ndcg = model_performance["sw"]["ndcg"].mean()
so_ndcg = model_performance["so"]["ndcg"].mean()
print("StreamWalk NDCG:", sw_ndcg)
print("Online second order NDCG:", so_ndcg)

### b.) Combination of StreamWalk and online second order similarity

- StreamWalk and online second order similarity can learn different network properties from the edge stream. 
- We combined the output of StreamWalk and second order similarity by using the weighted average of the corresponding inner products as similarity. 

In [ ]:
from online_node2vec.evaluation.ndcg_computer import parallel_combined_eval_ndcg
from tqdm import tqdm

weights = list(np.arange(0.1,1.1,0.1))

# Execution takes more time!
combination_performance = []
for second_order_weight in tqdm(weights):
    combination_performance.append(pd.concat(parallel_combined_eval_ndcg(
        feature_sets["so"], 
        feature_sets["sw"], 
        second_order_weight, 
        gen_id_to_account, 
        metric, 
        n_threads=max_threads
    )))

In [ ]:
mean_ndcg = [sw_ndcg] + [perf["ndcg"].mean() for perf in combination_performance] + [so_ndcg]
so_weights = [0.0] + weights + [1.0]

#### Combination on the first 3 days

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(so_weights, mean_ndcg)
plt.title(dataset_id.upper())
plt.xlabel("Weight of the second order model in the combination")
plt.ylabel("NDCG@100")
plt.show()

### c.) Results for the complete RG17 and UO17 datasets

![](http://info.ilab.sztaki.hu/~fberes/tennis/performance_table.png)

![](http://info.ilab.sztaki.hu/~fberes/tennis/temporal_performance.png)

<img src="http://info.ilab.sztaki.hu/~fberes/tennis/half_life.png" alt="drawing" width="585"/>

<img src="http://info.ilab.sztaki.hu/~fberes/tennis/number_of_walks_and_hash_functions.png" alt="drawing" width="570"/>

![](http://info.ilab.sztaki.hu/~fberes/tennis/combination.png)